# Data Collection and Storage

This notebook illustrates the collection and storage of data used for the project application.

Contents
--------
1. [Fetching data through API calls](#api)
    1. [Get geocoding information](#geocoding)
    2. [Get weather forecast](#weather)
2. [Web scraping](#scraping)
    1. [Reverse engineering the website requests](#scraping_reverse)
    2. [Get hotel information by scraping](#scraping_fetch)
3. [Storage in a data lake](#datalake)
4. [Storage in a database](#database)



## <a id="api"></a>API Calls

We request APIs to get geocoding information from the name of a place and weather forecast at given geographic coordinates. Utilities for the corresponding API calls are defined in the module `etl/api_mgmt.py`. To use them, we first setup a `requests.Session`. Since the API servers may limit the number of allowed requests, we add a retry policy to the HTTP session. We also load a file containing the names of the locations of interest.

In [ ]:
import csv
import json
import os
import shutil
import time

import numpy as np
import requests
from requests.adapters import HTTPAdapter, Retry

from etl import get_coords, get_weather_forecast, save_to_json

BUCKET_NAME = './bucket_name.key'
S3_WRITER_ACCESS_KEYS = './certification-project-s3-writer_accessKeys.key'
S3_READER_ACCESS_KEYS = './certification-project-s3-reader_accessKeys.key'
DB_ACCESS_KEYS = './db_access_keys.key'

In [2]:
# setup session with retry policy in case of failure
s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[403, 502, 503, 504])
s.mount('https://', HTTPAdapter(max_retries=retries))

# Load locations of interest
with open("./data/places.csv", 'rt', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader, None) # remove header
    locations = [row for row in reader]

### <a id="geocoding"></a>Get geocoding information

We use [Nominatim API](https://nominatim.org/) to fetch geocoding information. The API is quite restrictive in its [use policy](https://operations.osmfoundation.org/policies/nominatim/). Most importantly, the number of requests is limited to one per second, which forces us to throttle the rate accordingly.

In [3]:
coordinates = {}
for i, loc in enumerate(locations, start=1):
    coordinates[i] = ({'name': loc[0], 'country': loc[1]} 
                      | get_coords(s, f"{loc[0]}, {loc[1]}"))
    time.sleep(1.1)

coordinates[5]

{'name': 'Rouen',
 'country': 'France',
 'latitude': 49.4404591,
 'longitude': 1.0939658}

In [4]:
# save to csv
locations_cols = ['location_id', 'name', 'country', 'latitude', 'longitude']
with open('./data/locations.csv', 'wt', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_STRINGS)
    writer.writerow(locations_cols)
    for i, coords in coordinates.items():
        writer.writerow([i] + [coords[col] for col in locations_cols[1:]])

### <a id="weather"></a>Get weather forecast

We use [Open-Meteo API](https://open-meteo.com/en/docs) to get weather forecast information.

In [5]:
weather_forecast = {}
for i, coords in coordinates.items():
    weather_forecast[i] = get_weather_forecast(
        s, coords['latitude'], coords['longitude'])

weather_forecast[5]

{'latitude': 49.44,
 'longitude': 1.0999999,
 'generationtime_ms': 0.20241737365722656,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/Paris',
 'timezone_abbreviation': 'GMT+1',
 'elevation': 24.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_max': '°C',
  'temperature_2m_min': '°C',
  'sunshine_duration': 's',
  'precipitation_sum': 'mm'},
 'daily': {'time': ['2025-02-24',
   '2025-02-25',
   '2025-02-26',
   '2025-02-27',
   '2025-02-28',
   '2025-03-01',
   '2025-03-02',
   '2025-03-03',
   '2025-03-04',
   '2025-03-05',
   '2025-03-06',
   '2025-03-07',
   '2025-03-08',
   '2025-03-09',
   '2025-03-10',
   '2025-03-11'],
  'temperature_2m_max': [13.6,
   9.4,
   10.2,
   9.1,
   9.7,
   8.8,
   9.9,
   9.8,
   12.3,
   17.2,
   15.8,
   17.6,
   12.2,
   9.6,
   11.2,
   9.1],
  'temperature_2m_min': [9.7,
   5.9,
   4.9,
   3.5,
   1.2,
   0.6,
   0.2,
   0.9,
   2.4,
   6.8,
   7.1,
   7.3,
   7.0,
   5.4,
   5.3,
   2.7],
  'sunshine_duration': [10052.75,
   0.0,
   

In [6]:
# save to csv
# take the average weather over the next 7 days
weather_cols = [
    'location_id', 'date', 'min_temperature_cels', 'max_temperature_cels',
    'sunshine_duration_h', 'precipitation_sum_mm'
]
with open('./data/weather_indicators.csv', 'wt', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_STRINGS)
    writer.writerow(weather_cols)
    for i, forecast in weather_forecast.items():
        forecast = forecast['daily']
        row = [i, forecast['time'][0],
               np.mean(forecast['temperature_2m_min'][:8]),
               np.mean(forecast['temperature_2m_max'][:8]),
               np.mean(forecast['sunshine_duration'][:8])/3600,
               np.mean(forecast['precipitation_sum'][:8])]
        writer.writerow(row)

## <a id="scraping"></a>Web scraping

The collection of hotels information at the selected locations is done through web scraping of [booking.com](https://www.booking.com). This approach is more complex and unstable than API calls. It is standard practice to first study how requests and responses are related to web browser interaction. This will allow us to tailor automated requests for scraping.

### <a id="scraping_reverse"></a>Reverse engineering the website requests

The first step is to get the request resulting from regular user interaction. We thus go to the index page [https://www.booking.com/index.en-gb.html](https://www.booking.com/index.en-gb.html) and fill the search bar. Here we look for an hotal in Rouen, France for 2 adults between march, 1st and march, 9th.

<img src="media/booking_search.png" alt="booking_search" width="1000"/>

After clicking on "Search" the request actually sent by the web browser is displayed in the address bar. This allows us to recover the parameters of the GET request.

In [7]:
req_url = 'https://www.booking.com/searchresults.en-gb.html?ss=Rouen%2C+France&efdco=1&label=gen173nr-1FCAEoggI46AdICVgEaE2IAQGYAQm4ARjIAQ_YAQHoAQH4AQKIAgGoAgS4AsWf4r0GwAIB0gIkYmMwNmI1MjktMzkyZS00N2FjLTllNWYtOWZmZGIwMWZjODhj2AIF4AIB&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=-1462807&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=a6f166e2b250077a&ac_meta=GhBhNmYxNjZlMmIyNTAwNzdhIAAoATICZW46DVJvdWVuLCBGcmFuY2VAAEoAUAA%3D&checkin=2025-03-01&checkout=2025-03-09&group_adults=2&no_rooms=1&group_children=0'

req_url.split('?')[1].split('&')

['ss=Rouen%2C+France',
 'efdco=1',
 'label=gen173nr-1FCAEoggI46AdICVgEaE2IAQGYAQm4ARjIAQ_YAQHoAQH4AQKIAgGoAgS4AsWf4r0GwAIB0gIkYmMwNmI1MjktMzkyZS00N2FjLTllNWYtOWZmZGIwMWZjODhj2AIF4AIB',
 'aid=304142',
 'lang=en-gb',
 'sb=1',
 'src_elem=sb',
 'src=index',
 'dest_id=-1462807',
 'dest_type=city',
 'ac_position=0',
 'ac_click_type=b',
 'ac_langcode=en',
 'ac_suggestion_list_length=5',
 'search_selected=true',
 'search_pageview_id=a6f166e2b250077a',
 'ac_meta=GhBhNmYxNjZlMmIyNTAwNzdhIAAoATICZW46DVJvdWVuLCBGcmFuY2VAAEoAUAA%3D',
 'checkin=2025-03-01',
 'checkout=2025-03-09',
 'group_adults=2',
 'no_rooms=1',
 'group_children=0']

We can already get a few insights from the request URL:
- `ss` (search string) corresponds to the text written in the search bar,
- `checkin` and `checkout` correspond to the travel dates (the calendar widget in the center),
- `group_adults`, `no_rooms` and `group_children` correspond to the input of the right widget.

It turns out that a valid request can be made with a different approach, by specifying only the latitude and longitude of the destination. For instance, the URL
`'https://www.booking.com/searchresults.en-gb.html?latitude=49.4404591&longitude=1.0939658'` yields a page with the hotels ranked by inreasing distance to the coordinates. The parameters above can also be specified to refine the search, but we will not use them here.

### <a id="scraping_fetch"></a>Get hotel information by scraping

The above analysis helped to setup the scraping functionality, which is implemented in the module `etl/scraping_mgmt.py`. Let us detail briefly the sraping procedure:
- We reach the target website through an automated browser driven with  [Selenium](https://www.selenium.dev/) WebDriver. We favor this approach over other possibilities such as using `scrapy`. The reason is that the page loading of our target, booking.com, is mostly done through scrolling. This feature is implemented in javascript and is complex to trigger if the scraping tool used cannot execute javascript. A browser, however, natively executes javascript and therefore suits better our task.
- For each location, we send a request with the URL `'https://www.booking.com/searchresults.en-gb.html?latitude={latitude}&longitude={longitude}'`.
- We scrape the hotels data that we need, scrolling down if necessary.

In [8]:
from selenium import webdriver

from etl import scrape_from_searchpage

In [9]:
## setup driver with options to prevent detection
## See https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec/53040904#53040904
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd(
    'Network.setUserAgentOverride',
    {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'}
)
driver.implicitly_wait(1)

In [10]:
with open("./data/locations.csv", 'rt', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';', quoting=csv.QUOTE_STRINGS)
    next(reader, None) # remove header
    locations = [row for row in reader]

search_urls = {int(loc[0]): ('https://www.booking.com/searchresults.en-gb.html?'
                            f'latitude={loc[3]}&longitude={loc[4]}')
               for loc in locations}

for i, search_url in search_urls.items():
    hotel_infos = {i: scrape_from_searchpage(driver, search_url, limit=30)} # scrape on search
    save_to_json(f'./data/temp/{i}.json', hotel_infos)

driver.quit()

In [11]:
# load temporary saved files
hotels_list = []
root, _, files = next(os.walk('./data/temp/'))
for file in files:
    with open(root + file, 'rt', encoding='utf-8') as f:
        hotels_list.append(json.load(f))

# transform and save csv
hotels_cols = ['hotel_id', 'location_id', 'url', 'name',
               'description', 'rating', 'georating']
hotel_id = 1
hotels_data = []
for hotels in hotels_list:
    loc_id, hotels = next(iter(hotels.items()))
    for h in hotels:
        entry = [hotel_id, int(loc_id)] + [h[col] for col in hotels_cols[2:]]
        hotels_data.append(entry)
        hotel_id += 1

with open('./data/hotels.csv', 'wt', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_STRINGS)
    writer.writerow(hotels_cols)
    for row in hotels_data:
        writer.writerow(row)

In [12]:
## remove temp directory
shutil.rmtree('./data/temp/')

## <a id="datalake"></a>Storage in a data lake

For long-term storage, the collected data is transferred a data lake in csv format. We use an AWS S3 bucket for that purpose. The interaction with an S3 bucket through `boto3` is simple enough so that no wrappers need to be implemented for uploading. The module `etl/s3_mgmt.py` implements only a function for downloading and formatting the csv into a structure suitable for transfer to the database. The data is stored as multiple csv files matching the database structure.

In [13]:
import boto3

In [14]:
## create S3 client with writing permission
with open(BUCKET_NAME, 'rt', encoding='utf-8') as f:
    bucket_name = f.read()
with open(S3_WRITER_ACCESS_KEYS, 'rt', encoding='utf-8') as f:
    aws_access_key_id, aws_secret_access_key = f.readlines()[-1].strip().split(',')

s3_writer = boto3.client('s3', # region_name=region_name,
                         aws_access_key_id=aws_access_key_id, 
                         aws_secret_access_key=aws_secret_access_key)

In [15]:
## Uplpoad the files created
s3_writer.upload_file('./data/locations.csv', Bucket=bucket_name, Key='data/locations.csv')
s3_writer.upload_file('./data/weather_indicators.csv', Bucket=bucket_name, Key='data/weather_indicators.csv')
s3_writer.upload_file('./data/hotels.csv', Bucket=bucket_name, Key='data/hotels.csv')

The uploaded data can be inspected directly from the AWS web interface, as shown in the screenshot below.

<img src="media/S3_storage.png" alt="AWS S3 storage" width="1000"/>

## <a id="database"></a>Storage in a database

We use a database to load data for the application. The functionality to transfer and load data from the database is implemented in the module `etl/db_mgmt.py`. We use SQLAlchemy to define our database structure, connected to a PostgreSQL database hosted on [Neon](https://neon.tech). The structure of the database is the following.

<img src="media/DB_structure.png" alt="Database structure" width="700"/>

In [16]:
from sqlalchemy import create_engine, URL, inspect
from sqlalchemy import select
from sqlalchemy.orm import Session

from etl import download_csv
from etl import Base, Location, Hotel, WeatherIndicator

In [ ]:
## get credentials
with open(DB_ACCESS_KEYS, 'rt', encoding='utf-8') as f:
    pghost = f.readline().split("'")[1]
    pgdatabase = f.readline().split("'")[1]
    pguser = f.readline().split("'")[1]
    pgpassword = f.readline().split("'")[1]

url = URL.create(
    "postgresql+psycopg",
    username=pguser,
    password=pgpassword,
    host=pghost,
    database=pgdatabase,
)

## setup SQL engine
engine = create_engine(url, echo=False)
# inspector = inspect(engine)

## clear the database
# Base.metadata.drop_all(engine)

## create the database
Base.metadata.create_all(engine)

In [18]:
## create S3 client with only readig permission
with open(S3_READER_ACCESS_KEYS, 'rt', encoding='utf-8') as f:
    aws_access_key_id, aws_secret_access_key = f.readlines()[-1].strip().split(',')

s3_reader = boto3.client('s3', # region_name=region_name,
                         aws_access_key_id=aws_access_key_id, 
                         aws_secret_access_key=aws_secret_access_key)

In [19]:
## Load data from S3 buckets
locations = download_csv(s3_reader, bucket_name, 'data/locations.csv')
locations = [Location(**d) for d in locations]

weather_indicators = download_csv(s3_reader, bucket_name, 'data/weather_indicators.csv')
weather_indicators = [WeatherIndicator(**d) for d in weather_indicators]

hotels = download_csv(s3_reader, bucket_name, 'data/hotels.csv')
hotels = [Hotel(**d) for d in hotels]

In [20]:
## Transfer to database
Base.metadata.create_all(engine)
with Session(engine) as session:
    session.add_all(locations)
    session.commit()
    session.add_all(weather_indicators)
    session.commit()
    session.add_all(hotels)
    session.commit()

The transferred data can be seen on the server, as shown below.

<img src="media/DB_storage.png" alt="Database structure" width="1200"/>

Finally, we can load the database contents directly into a pandas `DataFrame`, which is how we will load the data to display in the application.

In [21]:
## read database table directly into dataframe
import pandas as pd

pd.read_sql(select(Location.__table__), con=engine)

,location_id,name,country,latitude,longitude
0,1,Mont Saint Michel,France,48.635954,-1.511460
1,2,St Malo,France,48.649518,-2.026041
2,3,Bayeux,France,49.276462,-0.702474
3,4,Le Havre,France,49.493898,0.107973
4,5,Rouen,France,49.440459,1.093966
5,6,Paris,France,48.858890,2.320041
6,7,Amiens,France,49.894171,2.295695
7,8,Lille,France,50.636565,3.063528
8,9,Strasbourg,France,48.584614,7.750713
9,10,Chateau du Haut Koenigsbourg,France,48.249411,7.344320


In [22]:
## close database connection gracefully
engine.dispose()